In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import re
import random as rd
import torch
from torchvision import ops

import cv2
import os
import seaborn as sns

from ast import literal_eval
from google.colab.patches import cv2_imshow

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



In [ ]:
def ord(a, b):
  if a > b:
    x_ = a
    a = b
    b = x_
    return a , b
  else:
    return a, b



def make_rectangle(xmax_,xmin_,ymax_,ymin_):

    x1 = rd.uniform(0,2703)
    x2 = rd.uniform(0,2073)
    x1, x2 = ord(x1, x2)
    # print("X1 e x2:  ",x1,x2)
    while( (x2 - x1 > xmax_) or (x2 - x1 < xmin_)) :
      x2 = rd.uniform(0,2073)
      # print("Novo x2;  ",x2)
      x1, x2 = ord(x1, x2)
      # print("Novo X1 e X2;  ",x1, x2)


    y1 = rd.uniform(0,1519)
    y2 = rd.uniform(0,1519)
    y1, y2 = ord(y1, y2)
    # print("Y1 e Y2:  ",x1,x2)
    while( (y2 - y1 > ymax_) or (y2 - y1 < ymin_)):
      y2 = rd.uniform(0,1519)
      # print("Novo Y2;  ",y2)
      y1, y2 = ord(y1, y2)
      # print("Novo Y1 e Y2;  ",y1, y2)


    # print("X :")
    # print(x1, x2)
    # print(abs(x2 - x1))
    # print("Y :")
    # print(y1, y2)
    # print(abs(y2 - y1))

    return x1,x2,y1,y2


# a1 = 205.4000000000001 # max de x2 - x1
# b1 = 397.70000000000005 # max de y2 - y1
# a2 = 4.7999999999999545
# b2 = 5.899999999999864
# c1,c2,d1,d2 = make_rectangle(a1,a2,b1,b2)




# paramentros

In [ ]:
# folga na area das anotações
space = 1
# HOG paramentros
cell_size = 8
cell_block = 2
hog_w_size = 64
hog_h_size = 80
# area do roi
w_param = 1.4
h_param = 1.4
iou_threshold = 0.5
# Hough
param = 1.5
min_dis = 20
val1 = 40
val2 = 20

# Donwload das Imagens

In [ ]:
!git clone https://github.com/andersonzambeli/visao_computacional.git images

In [ ]:
img = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/00005.jpg')
display(img[...,::-1])
plt.imshow(img[...,::-1])
plt.show()
print(img.shape)

# Pega os as informações dos box das imagens de treino

In [ ]:
with open("/content/drive/MyDrive/archive/train_dataset/train.json") as f:
    data_dict = json.load(f)

# Create a list containing the data rows of the table
data = []
files_ = sorted(os.listdir(path = "/content/drive/MyDrive/archive/train_dataset/train_images"))
print(files_)
print(len(files_))

not_num = 0

# Loop through the elements in the annotations list
for annotation in data_dict['annotations']:
    # Get general bounding box information
    filename = annotation['filename']
    temp_str = re.sub('train_images\\\\', '', filename)
    if(temp_str not in files_):
       continue

    # if annotation["ignore"]:
    #   ignore = annotation["ignore"]
    #   if ignore == 1:
    #     continue
    xmin = annotation['bndbox']['xmin']
    ymin = annotation['bndbox']['ymin']
    xmax = annotation['bndbox']['xmax']
    ymax = annotation['bndbox']['ymax']

    if annotation['inbox']:

        for inbox in annotation['inbox']:
            color = inbox['color']
            if color == "yellow":
              continue
            shape = inbox['shape']
            # if (shape == str(-1) or shape == str(0)):
            if (True):
              not_num += 1

              data.append({
                      'filename': filename,
                      'xmin': xmin,
                      'ymin': ymin,
                      'xmax': xmax,
                      'ymax': ymax,
                      'color': color,
                  })




# Create a DataFrame from a list of data and save it to a CSV file
print(data)
df = pd.DataFrame(data)
df['filename'] = df['filename'].str.replace('train_images\\\\', '', regex=True)
df.to_csv('traffic_lights.csv', index=False)
print(df['color'].unique())
df.head()
print(not_num)

Vizualização de uma imagem com os bboxes

In [ ]:
img_name = "00051.jpg"

df_img_show = df.loc[df['filename'] == img_name]
df_img_show = df_img_show.reset_index()
print(df_img_show)

img = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/' + img_name)

for i in range(len(df_img_show.index)):
  x1b = df_img_show.at[i,"xmin"]
  x2b = df_img_show.at[i,"xmax"]
  y1b = df_img_show.at[i,"ymin"]
  y2b = df_img_show.at[i,"ymax"]
  img = cv2.rectangle(img, (int(x1b), int(y1b)), (int(x2b), int(y2b)), (0, 0, 255), 2)
  cv2.putText(img, df_img_show.at[i,"color"] , (int(x1b), int(y1b-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)


cv2_imshow(img)


Dividindo o dataset em parte de treino e parte de teste

In [ ]:
print(len(files_))

rd.shuffle(files_)

files_train = files_[0:600]
print(len(files_train))
files_test = files_[601:701]
print(len(files_test))

df_train = df[0:600]
print(df_train.shape)

# Treinando um algoritmo usando HOG com as bound boxes das imagens de treinamento

Contagem do numero de boxes de cada classe(green ou red)

In [ ]:
reds=0
greens=0
for i in range(len(df_train.index)):
  if df.at[i,"color"] == "red":
    reds += 1
  if df.at[i,"color"] == "green":
    greens += 1
print("Reds: ", reds)
print("Greens: ", greens)
print("Total: ", str(reds+greens))



Achando o maior valor de distancia em x e y para cada bound box e a mesma coisa para y

In [ ]:
#usar proporção TODO

xmax = 0
ymax = 0
xmin = 3000
ymin = 3000
for i in range(len(df_train.index)):
  x_dist = df.at[i,"xmax"] - df.at[i,"xmin"]
  y_dist = df.at[i,"ymax"] - df.at[i,"ymin"]
  if x_dist > xmax:
    xmax = x_dist
  if x_dist < xmin:
    xmin = x_dist
  if y_dist > ymax:
    ymax = y_dist
  if y_dist < ymin:
    ymin = y_dist

rz1 = ymax / xmax
rz2 = ymin / xmin
rz_opt = 1.5

print(rz1)
print(rz2)
print(xmax, ymax, xmin, ymin)

Pra cada imagem de treino, vamos gerar um número maximo de bound box em algum lugar fora das bound boxes de semáforo que seja igual ao numero de bound boxes de semáforo. Assim o algoritmo consgue saber o que não é semáfaro

In [ ]:

k = len(df_train.index)
max = 150
count = 0
for i in range(k):
  # print('')
  # print("Para i : " + str(i))
  if i == k:
    break
  df_file = df_train.loc[df_train['filename'] == df_train.at[i,"filename"]]
  filename_ = df_train.at[i,"filename"]
  df_file = df_file.reset_index()
  # print("Nome da imagem: " + filename_)
  if count > max:
        break


  x1,x2,y1,y2 = make_rectangle(xmax,xmin,ymax,ymin)
  # print("Rec criado",x1,x2,y1,y2)
  # img_test=img.copy()
  if i < k:
    for j in range(len(df_file.index)):
      if j == 0:
        img_test = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/' + df_file.at[j,"filename"])
      x1b = df_file.at[j,"xmin"]
      x2b = df_file.at[j,"xmax"]
      y1b = df_file.at[j,"ymin"]
      y2b = df_file.at[j,"ymax"]
      img_test = cv2.rectangle(img_test, (int(x1b), int(y1b)), (int(x2b), int(y2b)), (0, 0, 255), 2)
      bbox_rec = torch.tensor([[x1,y1,x2,y2]], dtype=torch.float)
      bbox_file = torch.tensor([[x1b,y1b,x2b,y2b]], dtype=torch.float)
      iou = ops.box_iou(bbox_file, bbox_rec)
      # print("Iou ", str(j)," :",iou.numpy()[0][0], )

      # 1.9362220058422583 1
      # 1.2291666666666499 2

      while((iou.numpy()[0][0] != 0) and (y2/x2 > rz1 or y2/x2 < rz2 )):

        x1,x2,y1,y2 = make_rectangle(xmax,xmin,ymax,ymin)
        # print("Novo Rec criado: ",x1,x2,y1,y2)
        bbox_rec = torch.tensor([[x1,y1,x2,y2]], dtype=torch.float)
        iou = ops.box_iou(bbox_file, bbox_rec)
        # print("Recalculando iou: " + str(j) )
        # (print("Novo Iou ", str(j)," :" ,iou.numpy()[0][0] ))
        # print(i)


    df2 = {'filename': filename_, 'xmin': x1, 'ymin': y1,'xmax': x2, 'ymax': y2,'color': "None"}
    count += 1
    df_train = df_train._append(df2, ignore_index = True)
    img_test = cv2.rectangle(img_test, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
    # cv2_imshow(img_test)
    df_file = df_train.loc[df_train['filename'] == df_train.at[i,"filename"]]
    # print(df_train.tail())












Processando as imagens da bbox

In [ ]:
from skimage import feature

print(df_train.shape)

img_ind = 380
print(files_train[img_ind])

img = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/' + files_train[img_ind] )



image = cv2.GaussianBlur(img, (3, 3), 0)
cv2_imshow(image)



# Retinex

Aplicação do retinex

In [ ]:
def singleScaleRetinex(img,variance):
    retinex = np.log10(img) - np.log10(cv2.GaussianBlur(img, (0, 0), variance))
    return retinex

def multiScaleRetinex(img, variance_list):
    retinex = np.zeros_like(img)
    for variance in variance_list:
        retinex += singleScaleRetinex(img, variance)
    retinex = retinex / len(variance_list)
    return retinex



def MSR(img, variance_list):
    img = np.float64(img) + 1.0
    img_retinex = multiScaleRetinex(img, variance_list)

    for i in range(img_retinex.shape[2]):
        unique, count = np.unique(np.int32(img_retinex[:, :, i] * 100), return_counts=True)
        for u, c in zip(unique, count):
            if u == 0:
                zero_count = c
                break
        low_val = unique[0] / 100.0
        high_val = unique[-1] / 100.0
        for u, c in zip(unique, count):
            if u < 0 and c < zero_count * 0.1:
                low_val = u / 100.0
            if u > 0 and c < zero_count * 0.1:
                high_val = u / 100.0
                break
        img_retinex[:, :, i] = np.maximum(np.minimum(img_retinex[:, :, i], high_val), low_val)

        img_retinex[:, :, i] = (img_retinex[:, :, i] - np.min(img_retinex[:, :, i])) / \
                               (np.max(img_retinex[:, :, i]) - np.min(img_retinex[:, :, i])) \
                               * 255
    img_retinex = np.uint8(img_retinex)
    return img_retinex



def SSR(img, variance):
    img = np.float64(img) + 1.0
    img_retinex = singleScaleRetinex(img, variance)
    for i in range(img_retinex.shape[2]):
        unique, count = np.unique(np.int32(img_retinex[:, :, i] * 100), return_counts=True)
        for u, c in zip(unique, count):
            if u == 0:
                zero_count = c
                break
        low_val = unique[0] / 100.0
        high_val = unique[-1] / 100.0
        for u, c in zip(unique, count):
            if u < 0 and c < zero_count * 0.1:
                low_val = u / 100.0
            if u > 0 and c < zero_count * 0.1:
                high_val = u / 100.0
                break
        img_retinex[:, :, i] = np.maximum(np.minimum(img_retinex[:, :, i], high_val), low_val)

        img_retinex[:, :, i] = (img_retinex[:, :, i] - np.min(img_retinex[:, :, i])) / \
                               (np.max(img_retinex[:, :, i]) - np.min(img_retinex[:, :, i])) \
                               * 255
    img_retinex = np.uint8(img_retinex)
    return img_retinex


variance_list=[15, 80, 30]
variance=300

In [ ]:


# img_ssr=SSR(img, variance)


# plt.imshow(img_ssr[...,::-1])
# plt.show()

# Segmentação por cor

**Conversão da imagem para HSV**

# Sem retinex

# Transformar para colormap para hsv

In [ ]:
img_hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)



# display(img_hsv[...,::1])
plt.imshow(img_hsv[...,::1])
plt.show()
# display(img_hsv[...,::-1])
plt.imshow(image[...,::-1])
plt.show()

# Aplicação de threshold de cores verde e vermelho e obtenção de aŕeas de interesse

Definimos o valores de threshold e criamos mascaras para achar regiões onde é verde e vermelho

Para o vermelho

In [ ]:
lower_red1 = np.array([0,100,100])
upper_red1 = np.array([8,255,255])
lower_red2 = np.array([160, 150, 90])
upper_red2 = np.array([180, 255, 255])

mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)

maskr = cv2.add(mask1, mask2)

# Fechamento
element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10,10))
closed_r  = cv2.morphologyEx(maskr, cv2.MORPH_CLOSE, element)
thresholded = closed_r
# (thresh1, thresholded) = cv2.threshold(closed_r, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)



# Calculos dos circulos de hogh


circles_red = cv2.HoughCircles(thresholded, cv2.HOUGH_GRADIENT, param, min_dis, param1=val1, param2= val2, minRadius=1, maxRadius=100)
NoneType = type(None)
if not isinstance(circles_red, NoneType):
  circles = np.uint16(np.around(circles_red))
  for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(closed_r,(i[0],i[1]),i[2],(255,0,0),2)
    # draw the center of the circle
    cv2.circle(closed_r,(i[0],i[1]),2,(0,0,255),3)

cv2_imshow(closed_r)

plt.imshow(closed_r[...,::1])
plt.show()



Para o verde

In [ ]:
lower_green = np.array([40, 120, 120])
upper_green = np.array([90, 255, 255])

maskg = cv2.inRange(img_hsv, lower_green, upper_green)



# Fechamento
closed_g  = cv2.morphologyEx(maskg, cv2.MORPH_CLOSE, element)
thresholded = closed_g
# (thresh1, thresholded) = cv2.threshold(closed_g, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#calculo do ciculo de hough
circles_green = cv2.HoughCircles(thresholded, cv2.HOUGH_GRADIENT, param, min_dis, param1=val1, param2=val2, minRadius=2, maxRadius=100)
NoneType = type(None)
if not isinstance(circles_green, NoneType):
  circles = np.uint16(np.around(circles_green))
  for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(closed_g,(i[0],i[1]),i[2],(255,0,0),2)
    # draw the center of the circle
    cv2.circle(closed_g,(i[0],i[1]),2,(0,0,255),3)

cv2_imshow(closed_g)

plt.imshow(closed_g[...,::-1])
plt.show()



gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
gradient_image = filters.sobel(gray_image)


Dataframe de Areas de interesse(ROI) da imagem

In [ ]:
image_anot = image.copy()
cv2_imshow(image_anot)

w_param = 1.3
h_param = 1.3
iou_threshold = 0.5

roi_df = pd.DataFrame()

if not isinstance(circles_red, NoneType):
  circles = np.uint16(np.around(circles_red))
  for r in circles[0,:]:

    xmin_r = r[0] - w_param*r[2]
    xmax_r = r[0] + w_param*r[2]
    ymin_r = r[1] - h_param*r[2]
    ymax_r = r[1] + 5 * h_param * r[2]


    roi_df = pd.concat([roi_df, pd.DataFrame([{"filename": img_name, "xmin": xmin_r , "ymin": ymin_r, "xmax": xmax_r,"ymax": ymax_r,"color": "None"}])], ignore_index=True)
    cv2.rectangle(image_anot, (int(xmin_r), int(ymin_r)), (int(xmax_r), int(ymax_r)), (0, 0, 255), 2)

    cv2.circle(image_anot,(r[0],r[1]),r[2],(255,0,0),2)
    cv2.circle(image_anot,(r[0],r[1]),r[2],(255,0,0),2)
    cv2.putText(image_anot, "red", (int(r[0]), int(r[1] - r[2] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    print(r)




if not isinstance(circles_green, NoneType):
  circles = np.uint16(np.around(circles_green))
  for r in circles[0,:]:

    xmin_r = r[0] - w_param*r[2]
    xmax_r = r[0] + w_param*r[2]
    ymin_r = r[1] - 5 * h_param * r[2]
    ymax_r = r[1] + h_param * r[2]

    roi_df = pd.concat([roi_df, pd.DataFrame([{"filename": img_name, "xmin": xmin_r , "ymin": ymin_r, "xmax": xmax_r,"ymax": ymax_r,"color": "None"}])], ignore_index=True)

    cv2.rectangle(image_anot, (int(xmin_r), int(ymin_r)), (int(xmax_r), int(ymax_r)), (0, 0, 255), 2)

    cv2.circle(image_anot,(r[0],r[1]),r[2],(255,0,0),2)
    cv2.putText(image_anot, "green", (int(r[0]), int(r[1] - r[2] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    print(r)

print(roi_df)

df_imag_anot = df_train.loc[df_train['filename'] == img_name]
df_imag_anot = df_imag_anot.reset_index()
print(df_imag_anot)

for i in range(len(df_imag_anot.index)):
  xmin_real = df_imag_anot.at[i,"xmin"]
  xmax_real = df_imag_anot.at[i,"xmax"]
  ymin_real = df_imag_anot.at[i,"ymin"]
  ymax_real = df_imag_anot.at[i,"ymax"]
  bbox_anot = torch.tensor([[xmin_real,ymin_real,xmax_real,ymax_real]], dtype=torch.float)


  for j in range(len(roi_df.index)):
    xmin_anot = roi_df.at[j,"xmin"]
    xmax_anot = roi_df.at[j,"xmax"]
    ymin_anot = roi_df.at[j,"ymin"]
    ymax_anot = roi_df.at[j,"ymax"]

    bbox_roi = torch.tensor([[xmin_anot,ymin_anot,xmax_anot,ymax_anot]], dtype=torch.float)
    iou1 = ops.box_iou(bbox_anot, bbox_roi)
    iou2 = ops.box_iou(bbox_roi,bbox_anot)

    if ((iou1 > iou_threshold) or (iou2> iou_threshold)):
      roi_df.at[j,"color"] = df_imag_anot.at[i,"color"]
    print("Iou i", str(i), " j ", str(j)," :",iou.numpy()[0][0], )


print(roi_df)

cv2_imshow(image_anot)

# Gerando bbox para cada círculo:


# HOG e treino do classifcador

In [ ]:
from skimage import feature
from skimage import exposure, filters



data_train = []
labels_train = []

for i in range(len(df_train.index)):
  img_name = df_train.at[i,"filename"]
  # print(img_name)



  #   break
  img = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/' + img_name, )
  image = cv2.GaussianBlur(img, (3, 3), 0)
  x1 = df_train.at[i,"xmin"]
  x2 = df_train.at[i,"xmax"]
  y1 = df_train.at[i,"ymin"]
  y2 = df_train.at[i,"ymax"]
  x1 = x1 - space*x1
  x2 = x2 + space*x1
  y1 = y1 - space*y1
  y2 = y2 + space*y1

  # print("Retangulo para cortar",x1,x2,y1,y2)

  cropped_image = img[int(y1):int(y2), int(x1):int(x2)]
  cropped_resized = cv2.resize(cropped_image, (hog_w_size,hog_h_size))

  gray_image = cv2.cvtColor(cropped_resized, cv2.COLOR_RGB2GRAY)
  # gradient_image = filters.sobel(gray_image)


  # cv2_imshow(cropped_image)
  # cv2_imshow(cropped_resized)
  # plt.imshow(gray_image, cmap='gray')
  # plt.show()
  features, hog_image = feature.hog(gray_image, orientations=9,
                               pixels_per_cell=(cell_size, cell_size), cells_per_block=(cell_block, cell_block),
                               visualize=True,
                               transform_sqrt=True, block_norm="L2-Hys")
  # plt.imshow(hog_image, cmap='gray')
  # plt.show()
  if (i == 1 or i == 0):
    print(img_name)
    cv2_imshow(cropped_image)
    cv2_imshow(cropped_resized)
    plt.imshow(hog_image, cmap='gray')
    plt.show()
  label = df_train.at[i,"color"]
  data_train.append(features)
  labels_train.append(label)
  # print(label)



Treinando o classificador

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

x_train = (np.array(data_train), np.array(labels_train))
print(len(labels_train))
print(len(data_train))


model = RandomForestClassifier(criterion='log_loss', max_depth=10)
# model = svm.SVC(decision_function_shape='ovo', probability=True)
# model =XGBClassifier()
X_train = np.array(data_train)
print(X_train.shape)
y_train = np.array(labels_train)
print(y_train.shape)

# # codifique os rótulos
le = LabelEncoder()
# codifique os rótulos de treino
y_train = le.fit_transform(y_train)
model.fit(X_train,y_train)



y_pred = model.predict(X_train)

cm=confusion_matrix(y_train,y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
print(classification_report(y_train,y_pred))








# Pegando o dataframe de anotações dos teste

In [ ]:

print(files_test)
print(len(files_test))

not_num = 0
data = []
# Loop through the elements in the annotations list
for annotation in data_dict['annotations']:
    # Get general bounding box information
    filename = annotation['filename']
    temp_str = re.sub('train_images\\\\', '', filename)
    # print(temp_str)

    # Pega somente as anotações das fotos da pasta atual
    if(temp_str not in files_):
       continue

    # if annotation["ignore"]:
    #   ignore = annotation["ignore"]
    #   if ignore == 1:
    #     continue
    xmin = annotation['bndbox']['xmin']
    ymin = annotation['bndbox']['ymin']
    xmax = annotation['bndbox']['xmax']
    ymax = annotation['bndbox']['ymax']

    if annotation['inbox']:

        for inbox in annotation['inbox']:
            color = inbox['color']
            if color == "yellow":
              continue
            shape = inbox['shape']
            # if (shape == str(-1) or shape == str(0)):
            if (True):
              not_num += 1

              data.append({
                      'filename': filename,
                      'xmin': xmin,
                      'ymin': ymin,
                      'xmax': xmax,
                      'ymax': ymax,
                      'color': color,
                  })




# Create a DataFrame from a list of data and save it to a CSV file
# print(data)
df_test = pd.DataFrame(data)
df_test['filename'] = df_test['filename'].str.replace('train_images\\\\', '', regex=True)
df_test.to_csv('traffic_lights_test.csv', index=False)
print(df_test)
# print(df_test['color'].unique())
# print(df_test.head())
# print(not_num)

# img_name = "00029.jpg"

# df_img_show = df_test.loc[df_test['filename'] == img_name]
# df_img_show = df_img_show.reset_index()
# # print(df_img_show)

# img = cv2.imread('/content/images/test_images/' + img_name)

# for i in range(len(df_img_show.index)):
#   x1t = df_img_show.at[i,"xmin"]
#   x2t = df_img_show.at[i,"xmax"]
#   y1t = df_img_show.at[i,"ymin"]
#   y2t = df_img_show.at[i,"ymax"]
#   color = df_img_show.at[i,"color"]
#   # img = cv2.rectangle(img, (int(x1t), int(y1t)), (int(x2t), int(y2t)), (0, 0, 255), 2)
#   # cv2.putText(img, str(i), (int(x1t), int(y1t-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

# cropped_image = img[int(y1t):int(y2t), int(x1t):int(x2t)]
# cropped_resized = cv2.resize(cropped_image, (32,64))

# cv2_imshow(cropped_image)
# cv2_imshow(cropped_resized)
# features, hog_image = feature.hog(cropped_resized, orientations=9,
#                               pixels_per_cell=(cell_size, cell_size), cells_per_block=(2, 2),
#                               visualize=True,
#                               transform_sqrt=True, block_norm="L2-Hys", multichannel=True)
# # plt.imshow(hog_image, cmap='gray')
# # plt.show()

# data=[]
# labels = []
# data.append(features)
# labels.append(color)

# x_test = np.array(data), np.array(labels)
# print(len(labels))
# print(len(data))
# pred = model.predict(x_test[0])
# print("Resultado: ", pred)

# # cv2_imshow(img)




# Gerando as Roi para todas as imagens

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:

data_roi=pd.DataFrame()

Iou_count = 0

for f in files_test:
  img_original = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/' + f)
  name_img = f
  # name_img = "02853.jpg"
  # img_original = cv2.imread('/content/drive/MyDrive/archive/train_dataset//train_images/' + name_img)

  # Passando filtro gaussiano:

  print(f)
  img_gauss_blur = cv2.GaussianBlur(img_original, (3, 3), 0)
  img_hsv=cv2.cvtColor(img_gauss_blur,cv2.COLOR_BGR2HSV)

  ########### Achando as rois vermelhas  ###########

  # Máscaras usadas
  lower_red1 = np.array([0,100,100])
  upper_red1 = np.array([8,255,255])
  lower_red2 = np.array([160, 150, 90])
  upper_red2 = np.array([180, 255, 255])

  #  Passandos as máscaras
  mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
  mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)

  maskr = cv2.add(mask1, mask2)

  # Fechamento das áreas geradas
  element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
  closed_r  = cv2.morphologyEx(maskr, cv2.MORPH_CLOSE, element)

  thresholded = closed_r
  # (thresh1, thresholded) = cv2.threshold(closed_r, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  # Calculos dos circulos de hogh
  circles_red = cv2.HoughCircles(thresholded, cv2.HOUGH_GRADIENT, param, min_dis, param1=40, param2=20, minRadius=2, maxRadius=100)
  NoneType = type(None)
  if not isinstance(circles_red, NoneType):
    circles = np.uint16(np.around(circles_red))
    for i in circles[0,:]:
      # contorno
      cv2.circle(closed_r,(i[0],i[1]),i[2],(255,0,0),2)
      # centro
      cv2.circle(closed_r,(i[0],i[1]),2,(0,0,255),3)



  ############ Gerando rois verdes ###########

  lower_green = np.array([40, 120, 120])
  upper_green = np.array([90, 255, 255])

  maskg = cv2.inRange(img_hsv, lower_green, upper_green)



# Fechamento
  closed_g  = cv2.morphologyEx(maskg, cv2.MORPH_CLOSE, element)

# OSTU
  thresholded = closed_g
  # (thresh1, thresholded) = cv2.threshold(closed_g, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#calculo do ciculo de hough
  circles_green = cv2.HoughCircles(thresholded, cv2.HOUGH_GRADIENT, param, min_dis, param1=val1, param2=val2, minRadius=1, maxRadius=100)
  NoneType = type(None)
  if not isinstance(circles_green, NoneType):
    circles = np.uint16(np.around(circles_green))
    for i in circles[0,:]:
      # draw the outer circle
      cv2.circle(closed_g,(i[0],i[1]),i[2],(255,0,0),2)
      # draw the center of the circle
      cv2.circle(closed_g,(i[0],i[1]),2,(0,0,255),3)






  #################### Criação das rois #################



  data_roi_loc = pd.DataFrame()

  image_anot = img_original.copy()
  # Para cor vermelha
  if not isinstance(circles_red, NoneType):
    circles = np.uint16(np.around(circles_red))
    for r in circles[0,:]:

      xmin_r = r[0] - w_param*r[2]
      if(xmin_r <= 0):
        xmin_r = 0
      xmax_r = r[0] + w_param*r[2]
      if(xmax_r >= 2703):
        xmax_r = 2703
      ymin_r = r[1] - h_param*r[2]
      if(ymin_r <= 0):
        ymin_r = 0
      ymax_r = r[1] + 5 * h_param * r[2]
      if(ymax_r >= 1519):
        yman_r = 1519
      if f == "00232.jpg":
        print(xmax_r,ymax_r,xmin_r,ymin_r)


      data_roi_loc = pd.concat([data_roi_loc, pd.DataFrame([{"filename": name_img, "xmin": xmin_r , "ymin": ymin_r, "xmax": xmax_r,"ymax": ymax_r,"color": "None"}])], ignore_index=True)
      cv2.rectangle(image_anot, (int(xmin_r), int(ymin_r)), (int(xmax_r), int(ymax_r)), (0, 0, 255), 2)

      cv2.circle(image_anot,(r[0],r[1]),r[2],(255,0,0),2)
      cv2.circle(image_anot,(r[0],r[1]),r[2],(255,0,0),2)
      cv2.putText(image_anot, "red", (int(r[0]), int(r[1] - r[2] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

      if f == "00232.jpg":
        print(r)



  # Para cor verde
  if not isinstance(circles_green, NoneType):
    circles = np.uint16(np.around(circles_green))
    for r in circles[0,:]:

      xmin_r = r[0] - w_param*r[2]
      if(xmin_r <= 0):
        xmin_r = 0
      xmax_r = r[0] + w_param*r[2]
      if(xmax_r >= 2703):
        xmax_r = 2703
      ymin_r = r[1] - 5 * h_param * r[2]
      if(ymin_r <= 0):
        ymin_r = 0
      ymax_r = r[1] + h_param * r[2]
      if(ymax_r >= 1519):
        ymax_r = 1519
      if f == "00232.jpg":
        print(xmax_r,ymax_r,xmin_r,ymin_r)

      data_roi_loc = pd.concat([data_roi_loc, pd.DataFrame([{"filename": name_img, "xmin": xmin_r , "ymin": ymin_r, "xmax": xmax_r,"ymax": ymax_r,"color": "None"}])], ignore_index=True)

      cv2.rectangle(image_anot, (int(xmin_r), int(ymin_r)), (int(xmax_r), int(ymax_r)), (0, 0, 255), 2)

      cv2.circle(image_anot,(r[0],r[1]),r[2],(255,0,0),2)
      cv2.putText(image_anot, "green", (int(r[0]), int(r[1] - r[2] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
      if f == "00232.jpg":
        print(r)



  # print(data_roi_loc)

  df_imag_anot = df_test.loc[df_test['filename'] == name_img]
  df_imag_anot = df_imag_anot.reset_index()
  # print(df_imag_anot)


  for i in range(len(df_imag_anot.index)):
    xmin_real = df_imag_anot.at[i,"xmin"]
    xmax_real = df_imag_anot.at[i,"xmax"]
    ymin_real = df_imag_anot.at[i,"ymin"]
    ymax_real = df_imag_anot.at[i,"ymax"]
    bbox_anot = torch.tensor([[xmin_real,ymin_real,xmax_real,ymax_real]], dtype=torch.float)


    for j in range(len(data_roi_loc.index)):
      xmin_anot = data_roi_loc.at[j,"xmin"]
      xmax_anot = data_roi_loc.at[j,"xmax"]
      ymin_anot = data_roi_loc.at[j,"ymin"]
      ymax_anot = data_roi_loc.at[j,"ymax"]

      bbox_roi = torch.tensor([[xmin_anot,ymin_anot,xmax_anot,ymax_anot]], dtype=torch.float)

      iou1 = ops.box_iou(bbox_roi, bbox_anot)
      iou2 = ops.box_iou(bbox_anot,bbox_roi)
      if ((iou1 > iou_threshold) or (iou2> iou_threshold)):
        data_roi_loc.at[j,"color"] = df_imag_anot.at[i,"color"]
      # print("Iou i", str(i), " j ", str(j)," :",iou.numpy()[0][0], )
      if iou > 0:
        Iou_count += 1





  # print(data_roi_loc)
  data_roi = pd.concat([data_roi, data_roi_loc], ignore_index=True)
  # print(data_roi)

  if f == "01082.jpg":
    cv2_imshow(img_original)

    cv2_imshow(closed_r)
    plt.imshow(closed_r[...,::1])
    plt.show()

    cv2_imshow(closed_g)
    plt.imshow(closed_g[...,::-1])
    plt.show()

    cv2_imshow(image_anot)
    print(data_roi_loc)
print(Iou_count)
print(data_roi)
print(data_roi.shape)














# Usando HOG nas imagens

In [ ]:
data = []
labels = []
for i in range(len(data_roi.index)):
  img_name = data_roi.at[i,"filename"]
  # print(img_name)


  #   break
  img = cv2.imread('/content/drive/MyDrive/archive/train_dataset/train_images/' + img_name, )
  image = cv2.GaussianBlur(img, (3, 3), 0)
  x1 = data_roi.at[i,"xmin"]
  x2 = data_roi.at[i,"xmax"]
  y1 = data_roi.at[i,"ymin"]
  y2 = data_roi.at[i,"ymax"]

  # print("Retangulo para cortar",x1,x2,y1,y2)

  cropped_image = image[int(y1):int(y2), int(x1):int(x2)]
  cropped_resized = cv2.resize(cropped_image, (hog_w_size,hog_h_size))

  gray_image = cv2.cvtColor(cropped_resized, cv2.COLOR_RGB2GRAY)

  features, hog_image = feature.hog(gray_image, orientations=9,
                               pixels_per_cell=(cell_size, cell_size), cells_per_block=(cell_block, cell_block),
                               visualize=True,
                               transform_sqrt=True, block_norm="L2-Hys")
  # plt.imshow(hog_image, cmap='gray')
  # plt.show()
  if (img_name == "02977.jpg"):
   cv2_imshow(cropped_image)
   cv2_imshow(cropped_resized)
   plt.imshow(hog_image, cmap='gray')
   plt.show()
   print(i)
  label = data_roi.at[i,"color"]
  data.append(features)
  labels.append(label)
  if i< 10:
    cv2_imshow(cropped_image)
    cv2_imshow(cropped_resized)
    plt.imshow(hog_image, cmap='gray')
    plt.show()
x_test = (np.array(data), np.array(labels))
print(len(labels))
print(len(data))

# print(labels)
# print(data)

# Passando pelo classificador

In [ ]:


X_test = np.array(data)
y_test = np.array(labels)

le = LabelEncoder()
# codifique os rótulos de treino
y_test = le.fit_transform(y_test)
y_pred = model.predict(X_test)

t = X_test[0].reshape(1,-1)
# print(t)
t_pred = model.predict(t)
print(t_pred)
# accuracy_test = sum(model.predict(X_test)==y_test)/len(y_test)
# print(accuracy_test)

cm=confusion_matrix(y_test,y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
print(classification_report(y_test,y_pred))